In [1]:
from pythonMD import manager
from unum.units import *
import matplotlib.pyplot as plt

# Test Lennard-Jones
## Two particles no thermostat

In [2]:
positions = [(4.,6.,6.),(8.,6.,6.),(6.,8.,8.)]
myManager = manager(forces=["LJ"],boundaries=[11.3,11.3,11.3],prop=[("VelocityVerlet",1.)],dimensions=3,cutoff=6*ANGSTROM,dt=1*fs)
myManager.initialize(positions=positions,masses=40*U,temperature=0*K,types=["Ar" for p in range(len(positions))])
data = myManager.run(10000,LJ={"Ar-Ar":{"sigma":3.4*ANGSTROM,"epsilon": 1.656778224E-21*J}},saveStats=1,savePositions=100,printStats=100)

100 1.3062265918058276e+18 8.111667135114189e-05 -0.000250518553556626 -0.00016940188220548413
200 1.8588990610871683e+18 0.00011543763169351314 -0.00028483895012991527 -0.00016940131843640212
300 1.4429431663066824e+18 8.960677062764499e-05 -0.00025900763140231797 -0.000169400860774673
400 9.89231769860678e+17 6.14312929083481e-05 -0.0002308320731869628 -0.0001694007802786147
500 7.10398322598907e+17 4.411573583339212e-05 -0.0002135165204484345 -0.00016940078461504237
600 5.97562829138973e+17 3.710865168953023e-05 -0.00020650946429530355 -0.0001694008126057733
700 5.781670500957222e+17 3.590417381094436e-05 -0.0002053050577801104 -0.00016940088396916604
800 4.898661917107108e+17 3.042069050523514e-05 -0.00019982169469934586 -0.00016940100419411073
900 1.6532094631832662e+17 1.0266430766368085e-05 -0.00017966729102818514 -0.00016940086026181705
1000 3.332480965160037e+16 2.069470679364383e-06 -0.00017147014102032428 -0.0001694006703409599
1100 4.122804980098752e+17 2.5602618926413254e-

In [4]:
xyz=myManager.makePositionsFile(data["positions"],"2particles.xyz")

## Increase timestep and check energy convergence

In [3]:
myManager = manager(forces=["LJ"],boundaries=[11.3,11.3,11.3],prop=[("VelocityVerlet",1.)],dimensions=3,cutoff=5*ANGSTROM)

# for dt in range(1,20,2):
#     myManager.dt = 0.01*dt
myManager.initialize(positions="coord_ex.xyz",masses=40*U,temperature=298*K)
data = myManager.run(600,LJ={"Ar-Ar":{"sigma":3.4*ANGSTROM,"epsilon": 1.656778224E-21*J}},saveStats=1,savePositions=1,printStats=10)
totalRelativeEnergy = [abs((energy-data["totalEnergy"][0])*100/energy) for energy in data["totalEnergy"]]
totalRelativeEnergy.sort()
# print(f"maximum change in energy for dt={dt}: {totalRelativeEnergy[-1]}%")

Reading 32 atoms
10 1.7554662314166008e+19 0.011628208316903563 -0.017196907534284527 -0.005568699217380964
20 1.7461926691249826e+19 0.011566780240283885 -0.017113288312644952 -0.005546508072361067
30 1.7303754109193945e+19 0.01146200672193007 -0.016966107865953862 -0.005504101144023792
40 1.707975985738734e+19 0.011313632929533374 -0.016754822886007867 -0.005441189956474492
50 1.6789825953490887e+19 0.011121580711592364 -0.016478435603461437 -0.005356854891869073
60 1.6436439225173273e+19 0.010887497342754777 -0.01613689905536609 -0.005249401712611313
70 1.6027730031276624e+19 0.010616768372717637 -0.015732872514524623 -0.005116104141806986
80 1.5581681305620318e+19 0.0103213056968429 -0.015309502608135616 -0.004988196911292715
90 1.5131744276963312e+19 0.010023267409060498 -0.014811635575575624 -0.004788368166515126
100 1.4718596335939582e+19 0.00974959821292638 -0.014330376787404743 -0.004580778574478363
110 1.4409553822939054e+19 0.00954488845231483 -0.013859204650314948 -0.004314

KeyboardInterrupt: 

In [10]:
xyz=myManager.makePositionsFile(data["positions"],"testArgon.xyz")

In [7]:
import numpy
numpy.floor([2.7,6.9])

array([2., 6.])